In [2]:
import csv
from serpapi import GoogleSearch
import time

# API 金鑰
API_KEY = "9be186a2a835a38c3da4f7548d16ea50fa97d60eaca5f5a19e095344440d6493"

# 儲存文章
articles = []

# 設置目標
q = '"Information technology capability"'
target = "Information technology capability"
# 最低引用次數
cited_by_info_qte = 200
# 搜尋頁數
page_num = 100

# 設置開始頁面
start = 0
while True:
    params = {
        "engine": "google_scholar",
        "q": q,
        "api_key": API_KEY,
        "hl": "en",
        "num": page_num,
        "start": start,
        "date_range": "2000-2024"
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # 檢查數據結構
    if "organic_results" not in results or not results["organic_results"]:
        break  # 如果沒有更多文章，停止請求

    for result in results.get("organic_results", []):
        title = result.get("title", "標題不可用")
        abstract = result.get("snippet", "")
        link = result.get("link", "")
        cited_by_info = result.get("inline_links", {}).get(
            "cited_by", {}).get("total", None)

        # 新的過濾條件：引用次數 > cited_by_info_qte 且標題或摘要中包含 target
        if cited_by_info is not None and cited_by_info > cited_by_info_qte and (target in title.lower() or target in abstract.lower()):
            articles.append(
                {"Title": title, "Cited By": cited_by_info, "link": link})  # 儲存標題和引用次數

    # 更新開始頁面
    start += page_num

    # 休眠以避免觸發 API 限制
    time.sleep(1)  # 根據需要調整休眠時間

# 將結果寫入 CSV 文件
file = target.replace(" ", "_")+".csv"
with open(file, "w", newline="", encoding="utf-8-sig") as csvfile:
    fieldnames = ["Title", "Cited By", "link"]  # 更新欄位名稱
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for article in articles:
        writer.writerow(article)

print(f"已將 {len(articles)} 篇符合條件的文章儲存到 {file}")

已將 0 篇符合條件的文章儲存到 Information_technology_capability.csv
